In [1]:
import pickle
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

%matplotlib widget

In [2]:
data_path = "data.pickle"

with open(data_path, "rb") as file:
    records = pickle.load(file)
    file.close()

In [3]:
dyad = '[100, 101]'
block = 1

rec = records[dyad][block]
rec = rec.iloc[0:120*60]  # use only the first 1 minute
rec_init = rec.iloc[0]

# Intialize the plot
fig, ax = plt.subplots()

# Agent trajectories
p0_agent, = ax.plot(rec_init['p0_agent_x'], rec_init['p0_agent_y'], marker='.', linewidth=0, markersize=20)
p1_agent, = ax.plot(rec_init['p1_agent_x'], rec_init['p1_agent_y'], marker='.', linewidth=0, markersize=20)
# Eye-tracking
p0_gaze, = ax.plot(rec_init['p0_gaze_x'], rec_init['p0_gaze_y'], marker='+', linewidth=0, markersize=10, color='tab:blue')
p1_gaze, = ax.plot(rec_init['p1_gaze_x'], rec_init['p1_gaze_y'], marker='+', linewidth=0, markersize=10, color='tab:orange')
# Targets
comp = plt.Circle(rec_init[['comp_x', 'comp_y']],    radius=0.05, color='black',  fill=False)
coop0 = plt.Circle(rec_init[['coop0_x', 'coop0_y']], radius=0.05, color='blue',   fill=True, alpha=0.5)
coop1 = plt.Circle(rec_init[['coop1_x', 'coop1_y']], radius=0.05, color='orange', fill=True, alpha=0.5)

ax.add_artist(comp)
ax.add_artist(coop0)
ax.add_artist(coop1)

ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_aspect(1)

# Function to update the plot for each frame
def update(frame):
    
    rec_frame = rec.iloc[frame]
    
    p0_agent.set_data(rec_frame['p0_agent_x'], rec_frame['p0_agent_y'])
    p1_agent.set_data(rec_frame['p1_agent_x'], rec_frame['p1_agent_y'])
    
    p0_gaze.set_data(rec_frame['p0_gaze_x'], rec_frame['p0_gaze_y'])
    p1_gaze.set_data(rec_frame['p1_gaze_x'], rec_frame['p1_gaze_y'])
    
    comp.set_center(rec_frame[['comp_x', 'comp_y']])
    coop0.set_center(rec_frame[['coop0_x', 'coop0_y']])
    coop1.set_center(rec_frame[['coop1_x', 'coop1_y']])

    return p0_agent, p1_agent, p0_gaze, p1_gaze, comp, coop0, coop1

# Create the animation
ani = FuncAnimation(fig, update, interval=8.33, frames=rec.shape[0], blit=True)
# plt.show()

# Save the animation in .mp4 format
writer = FFMpegWriter(fps=120)  # Adjust fps as needed
ani.save(f'replays/{dyad}_{block}.mp4', writer=writer)

plt.close()